In [2]:
# 데이터 불러오기
from dotenv import load_dotenv
import pandas as pd
import os 
import numpy as np
from sqlalchemy import create_engine

# .env 파일에서 환경 부르기
load_dotenv()

# 환경 변수 사용
database = os.getenv("CON_STR")


# # csv 파일 경로
# csv_file_path = os.getenv("CSV_FILE_PATH")
# print(csv_file_path)

engine = create_engine(database)


In [2]:

# ALL Raw Data
sql1= ''' 
select * from cu.agg_CU_TEREA_Total_Sourcing
where YYYYMM >= '202401'
order by YYYYMM, id
'''

gg = pd.read_sql_query(sql=sql1, con=engine)


In [16]:
gg.to_clipboard()

# 광역시/특별시 SIDO Pivot 작업

In [4]:
# - TEREA_flaXtar_ from 202211
sql2 = ''' 
select  
	t.YYYYMM,
	t.SIDO_NM,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) flavorXtar,
	count(distinct case when b.cigatype ='CC' then t.id end) CC,
	count(distinct case when b.cigatype ='HnB' then t.id end) HnB
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and b.CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'  
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where t.SIDO_NM in ('서울특별시','인천광역시','부산광역시','울산광역시','광주광역시','대전광역시', '대구광역시')
and t.YYYYMM >= '202401'
group BY 
	t.YYYYMM,
	t.SIDO_NM,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) 
'''

data2 = pd.read_sql_query(sql=sql2, con=engine)

pivot_flavor_tar = data2.pivot_table(index=['YYYYMM','SIDO_NM'], 
                                     columns='flavorXtar',
                                     values=['HnB', 'CC' ]
                                    )
print(pivot_flavor_tar)



                     CC                                                   \
flavorXtar     Fresh X  Fresh X 1MG Fresh X LTS Fresh X ULT New Taste X    
YYYYMM SIDO_NM                                                             
202401 광주광역시        0.0         7.0        10.0         NaN          0.0   
       대구광역시        0.0         7.0         4.0         1.0          0.0   
       대전광역시        0.0         4.0         6.0         2.0          0.0   
       부산광역시        0.0         4.0         7.0         2.0          0.0   
       서울특별시        0.0        32.0        27.0         4.0          0.0   
       울산광역시        0.0         3.0         4.0         1.0          0.0   
       인천광역시        0.0        12.0        14.0         2.0          0.0   
202402 광주광역시        0.0         9.0         5.0         NaN          0.0   
       대구광역시        0.0         6.0         4.0         1.0          0.0   
       대전광역시        0.0         9.0         4.0         NaN          0.0   
       부산광역시

In [5]:
pivot_flavor_tar.to_clipboard()

In [6]:
# - Terea_user_past_type_M1
sql = ''' 
select  
	t.YYYYMM, 
	t.SIDO_NM,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where t.SIDO_NM in ('서울특별시','인천광역시','부산광역시','울산광역시','광주광역시','대전광역시', '대구광역시')
and t.YYYYMM >= '202401'
group BY 	    
	t.YYYYMM, 
	t.SIDO_NM, 
	t.id
'''

data = pd.read_sql_query(sql=sql, con=engine)
# Past Type 
def categorize(row):
    categories = []
    if row['IQOS_Purchased'] == 1:
        categories.append('PMK HnB')
    if row['CC_Purchased'] == 1:
        categories.append('CC')
    if row['CompHnB_Purchased'] == 1:
        categories.append('Comp HnB')
    return ','.join(categories)

data['retype'] = data.apply(categorize, axis=1)
result = data.groupby(['YYYYMM','SIDO_NM', 'retype']).size().reset_index(name='N')
total = result['N'].sum()

pivot_past_type = result.pivot_table( 
                            index=['YYYYMM', 'SIDO_NM'],
                            columns='retype',
                            values='N'
                            )
print(pivot_past_type)



retype             CC  CC,Comp HnB  Comp HnB  PMK HnB  PMK HnB,CC  \
YYYYMM SIDO_NM                                                      
202401 광주광역시    131.0         23.0      15.0     27.0        17.0   
       대구광역시     96.0         24.0      12.0     16.0        11.0   
       대전광역시     87.0         22.0       4.0     21.0        11.0   
       부산광역시    136.0         28.0      19.0     24.0        22.0   
       서울특별시    373.0         75.0      52.0     76.0        48.0   
       울산광역시     99.0         27.0      11.0     16.0         9.0   
       인천광역시    163.0         24.0       9.0     33.0        23.0   
202402 광주광역시    112.0         32.0       9.0     22.0        13.0   
       대구광역시     99.0         14.0       9.0     17.0        18.0   
       대전광역시     98.0         22.0       5.0     18.0        12.0   
       부산광역시    159.0         29.0      15.0     34.0        18.0   
       서울특별시    360.0         70.0      31.0     89.0        38.0   
       울산광역시    113.0         24.0

In [7]:
pivot_past_type.to_clipboard()

In [8]:
sql3 = ''' 
with temp as (
select  
	t.YYYYMM, 
	t.SIDO_NM,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where t.SIDO_NM in ('서울특별시','인천광역시','부산광역시','울산광역시','광주광역시','대전광역시', '대구광역시')
and t.YYYYMM >= '202401'
group BY 	    	
	t.YYYYMM, 
	t.SIDO_NM,
	t.id
)
select YYYYMM, SIDO_NM,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
     as Cigatype,
    count(*) purchaser_cnt
from temp
group by YYYYMM, SIDO_NM,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

pivot_current_type = data3.pivot_table(index=['YYYYMM', 'SIDO_NM'],
                                       columns='Cigatype',
                                       values='purchaser_cnt',
                                       )

print(pivot_current_type)


Cigatype         IQOS  IQOS + CC  IQOS + Comp. HnB  IQOS + Comp. HnB + CC
YYYYMM SIDO_NM                                                           
202401 광주광역시     82.0      100.0              16.0                   20.0
       대구광역시     53.0       86.0               6.0                   16.0
       대전광역시     59.0       67.0               5.0                   14.0
       부산광역시     93.0      102.0              17.0                   26.0
       서울특별시    213.0      330.0              49.0                   48.0
       울산광역시     62.0       70.0              13.0                   17.0
       인천광역시     99.0      127.0              11.0                   18.0
202402 광주광역시     72.0       92.0               9.0                   18.0
       대구광역시     68.0       71.0               8.0                   13.0
       대전광역시     62.0       76.0               8.0                   12.0
       부산광역시     99.0      126.0              18.0                   17.0
       서울특별시    222.0      296.0      

In [9]:
pivot_current_type.to_clipboard()

# Gr Region Pivot 작업

In [10]:
# - TEREA_flaXtar_ from 202211
sql2 = ''' 
select  
	t.YYYYMM,
	gr_cd,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) flavorXtar,
	count(distinct case when b.cigatype ='CC' then t.id end) CC,
	count(distinct case when b.cigatype ='HnB' then t.id end) HnB
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and b.CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'  
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where t.YYYYMM >= '202401'
group BY 
	t.YYYYMM,
	gr_cd,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) 
'''

data2 = pd.read_sql_query(sql=sql2, con=engine)

pivot_flavor_tar = data2.pivot_table(index=['YYYYMM','gr_cd'], 
                                     columns='flavorXtar',
                                     values=['HnB', 'CC' ]
                                    )
print(pivot_flavor_tar)



                   CC                                                        \
flavorXtar   Fresh X  Fresh X 1MG Fresh X Below 1MG Fresh X LTS Fresh X ULT   
YYYYMM gr_cd                                                                  
202401 광주         0.0        24.0               NaN        29.0         5.0   
       기타         0.0        33.0               NaN        38.0        14.0   
       대구         0.0        18.0               NaN        15.0         5.0   
       대전         0.0        26.0               NaN        31.0         9.0   
       부산         0.0        25.0               NaN        25.0         4.0   
       서울         0.0       114.0               NaN       124.0        17.0   
202402 광주         0.0        32.0               NaN        27.0         4.0   
       기타         0.0        32.0               NaN        40.0         8.0   
       대구         0.0        19.0               NaN        13.0         4.0   
       대전         0.0        40.0               NaN 

In [11]:
pivot_flavor_tar.to_clipboard()

In [12]:
# - Terea_user_past_type_M1
sql = ''' 
select  
	t.YYYYMM, 
	gr_cd,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where t.YYYYMM >= '202401'
group BY 	    
	t.YYYYMM, 
	gr_cd, 
	t.id
'''

data = pd.read_sql_query(sql=sql, con=engine)
# Past Type 
def categorize(row):
    categories = []
    if row['IQOS_Purchased'] == 1:
        categories.append('PMK HnB')
    if row['CC_Purchased'] == 1:
        categories.append('CC')
    if row['CompHnB_Purchased'] == 1:
        categories.append('Comp HnB')
    return ','.join(categories)

data['retype'] = data.apply(categorize, axis=1)
result = data.groupby(['YYYYMM','gr_cd', 'retype']).size().reset_index(name='N')
total = result['N'].sum()

pivot_past_type = result.pivot_table( 
                            index=['YYYYMM', 'gr_cd'],
                            columns='retype',
                            values='N'
                            )
print(pivot_past_type)



retype            CC  CC,Comp HnB  Comp HnB  PMK HnB  PMK HnB,CC  \
YYYYMM gr_cd                                                       
202401 광주      490.0         84.0      42.0     91.0        62.0   
       기타      449.0        103.0      24.0     69.0        46.0   
       대구      340.0         61.0      37.0     39.0        31.0   
       대전      544.0         82.0      47.0     91.0        61.0   
       부산      556.0        106.0      51.0    103.0        64.0   
       서울     1481.0        252.0     140.0    289.0       195.0   
202402 광주      527.0         97.0      45.0     88.0        51.0   
       기타      476.0        106.0      37.0     60.0        45.0   
       대구      340.0         61.0      37.0     56.0        40.0   
       대전      564.0        115.0      41.0     77.0        59.0   
       부산      606.0        101.0      61.0     99.0        75.0   
       서울     1499.0        249.0     156.0    268.0       164.0   
202403 광주      510.0        112.0      48.0     

In [13]:
pivot_past_type.to_clipboard()

In [14]:
# - Terea_user_Current_type_M1

sql3 = ''' 
with temp as (
select  
	t.YYYYMM, 
	gr_cd,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from  cu.agg_CU_TEREA_Total_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where t.YYYYMM >= '202401'
group BY 	    	
	t.YYYYMM, 
	gr_cd,
	t.id
)
select YYYYMM, gr_cd,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
     as Cigatype,
    count(*) purchaser_cnt
from temp
group by YYYYMM, gr_cd,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
'''

data3 = pd.read_sql_query(sql=sql3, con=engine)

pivot_current_type = data3.pivot_table(index=['YYYYMM', 'gr_cd'],
                                       columns='Cigatype',
                                       values='purchaser_cnt',
                                       )

print(pivot_current_type)


Cigatype        IQOS  IQOS + CC  IQOS + Comp. HnB  IQOS + Comp. HnB + CC
YYYYMM gr_cd                                                            
202401 광주      296.0      360.0              47.0                   76.0
       기타      233.0      372.0              45.0                   59.0
       대구      162.0      279.0              27.0                   46.0
       대전      299.0      422.0              37.0                   77.0
       부산      330.0      424.0              48.0                   93.0
       서울      850.0     1234.0             133.0                  184.0
202402 광주      276.0      431.0              49.0                   62.0
       기타      212.0      411.0              33.0                   73.0
       대구      213.0      254.0              27.0                   46.0
       대전      304.0      456.0              51.0                   65.0
       부산      344.0      496.0              54.0                   68.0
       서울      816.0     1237.0             137.0  

In [15]:
pivot_current_type.to_clipboard()

# Pivot 작업 - Gr.지역 별 sourcing 작업해주신 내용에서 cc 에서 유입인 경우 cc family / subfamily / taste segment
## 1. CC Pivot

In [3]:
sql6 = ''' 
select  
	t.YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	FLAVORSEG_type3,
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd, FLAVORSEG_type3),
		(t.YYYYMM, FLAVORSEG_type3)
	)
'''

gr_Region_taste = pd.read_sql_query(sql=sql6, con=engine)


pivot_gr_taste = gr_Region_taste.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns=['FLAVORSEG_type3'],
                         values= 'purchasers'  
                        )

In [4]:
taste_sql = ''' 
with temp as (
	select t.YYYYMM, 
	gr_cd,
	t.id,
	max(b.FLAVORSEG_type3 ) + ' Only' FLAVORSEG_type3 
	from cu.agg_CU_TEREA_Total_Sourcing t
			join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM = t.YYYYMM	
		join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
		join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
	where t.YYYYMM >= '202401'
	group by t.YYYYMM, gr_cd, t.id
	having count(distinct b.FLAVORSEG_type3) = 1 
)
select YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	FLAVORSEG_type3 , 
	count(id) purchasers
from temp
group by 	
	grouping sets (
		(YYYYMM, gr_cd, FLAVORSEG_type3),
		(YYYYMM, FLAVORSEG_type3)
	) 
'''

gr_Region_taste_only = pd.read_sql_query(sql=taste_sql, con=engine)


pivot_gr_taste_only = gr_Region_taste_only.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns=['FLAVORSEG_type3'],
                         values= 'purchasers'  
                        )

In [5]:
# ProductFamilyCode

sql4 = ''' 
select  
	t.YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	ProductFamilyCode,
--	ProductSubFamilyCode,
--	FLAVORSEG_type3,
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM		
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd, ProductFamilyCode),
		(t.YYYYMM, ProductFamilyCode)
	)
'''

gr_Region_FamilyCode = pd.read_sql_query(sql=sql4, con=engine)


pivot_gr = gr_Region_FamilyCode.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns='ProductFamilyCode',
                         values= 'purchasers'  
                        )


In [6]:
# ProductSubFamilyCode

sql5 = ''' 
select  
	t.YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	ProductSubFamilyCode,
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd, ProductSubFamilyCode),
		(t.YYYYMM, ProductSubFamilyCode)
	)
'''

gr_Region_SubFamliyCode = pd.read_sql_query(sql=sql5, con=engine)


pivot_gr_SubFamliyCode = gr_Region_SubFamliyCode.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns=['ProductSubFamilyCode'],
                         values= 'purchasers'  
                        )


## 피벗 합치기 

In [7]:
sql7= ''' 
select  
	t.YYYYMM,  COALESCE(gr_cd, '합계') 'Gr Region',
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM = t.YYYYMM	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd),
		(t.YYYYMM)
	)
'''
gr_Region_Total = pd.read_sql_query(sql=sql7, con=engine)
gr_Region_Total = gr_Region_Total.pivot_table(index=['YYYYMM', 'Gr Region'],
                            values='purchasers')

ppp = pd.concat([gr_Region_Total, pivot_gr_taste, pivot_gr_taste_only, pivot_gr, pivot_gr_SubFamliyCode], axis=1)

ppp.to_clipboard()

## 2. 과거 3개월 동안 CC 구매 Pivot

In [8]:
cc_sql1 = ''' 
select
	t.YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	FLAVORSEG_type3,
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd, FLAVORSEG_type3),
		(t.YYYYMM, FLAVORSEG_type3)
	)
'''

gr_Region_cc_taste = pd.read_sql_query(sql=cc_sql1, con=engine)


pivot_gr_cc_taste = gr_Region_cc_taste.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns=['FLAVORSEG_type3'],
                         values= 'purchasers'  
                        )

In [9]:
cc_sql2 = ''' 
with temp as (
	select t.YYYYMM, 
			gr_cd, 
			t.id, 
			max(b.FLAVORSEG_type3 ) + ' Only' FLAVORSEG_type3 
	from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
							 AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)
		join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
		join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
	where t.YYYYMM >= '202401'
	group by t.YYYYMM, gr_cd, t.id
	having count(distinct b.FLAVORSEG_type3) = 1 
)
select YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	FLAVORSEG_type3 , 
	count(id) purchasers
from temp
group by 	
	grouping sets (
		(YYYYMM, gr_cd, FLAVORSEG_type3),
		(YYYYMM, FLAVORSEG_type3)
	) 
'''

gr_Region_CC_taste_only = pd.read_sql_query(sql=cc_sql2, con=engine)


pivot_gr_CC_taste_only = gr_Region_CC_taste_only.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns=['FLAVORSEG_type3'],
                         values= 'purchasers'  
                        )

In [10]:
# ProductFamilyCode

cc_sql3 = ''' 
select  
	t.YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	ProductFamilyCode,
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd, ProductFamilyCode),
		(t.YYYYMM, ProductFamilyCode)
	)
'''

gr_Region_CC_FamilyCode = pd.read_sql_query(sql=cc_sql3, con=engine)


pivot_gr_CC_FamilyCode = gr_Region_CC_FamilyCode.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns='ProductFamilyCode',
                         values= 'purchasers'  
                        )


In [11]:
# ProductSubFamilyCode

cc_sql4 = ''' 
select  
	t.YYYYMM, COALESCE(gr_cd, '합계') 'Gr Region',
	ProductSubFamilyCode,
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd, ProductSubFamilyCode),
		(t.YYYYMM, ProductSubFamilyCode)
	)
'''

gr_Region_CC_SubFamliyCode = pd.read_sql_query(sql=cc_sql4, con=engine)


pivot_gr_CC_SubFamliyCode = gr_Region_CC_SubFamliyCode.pivot_table(index=['YYYYMM', 'Gr Region'],
                         columns=['ProductSubFamilyCode'],
                         values= 'purchasers'  
                        )


In [12]:
Total_CC_sql= ''' 
select  
	t.YYYYMM,  COALESCE(gr_cd, '합계') 'Gr Region',
	count(distinct t.id) purchasers
from cu.agg_CU_TEREA_Total_Sourcing t
		join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
			and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
					 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' and b.cigatype='CC'
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
where 1=1
and t.YYYYMM >= '202401'
group BY
	grouping sets (
		(t.YYYYMM, gr_cd),
		(t.YYYYMM)
	)
'''
gr_Region_CC_3month = pd.read_sql_query(sql=Total_CC_sql, con=engine)
gr_Region_CC_3month = gr_Region_CC_3month.pivot_table(index=['YYYYMM', 'Gr Region'],
                            values='purchasers')

ppp = pd.concat([gr_Region_CC_3month, pivot_gr_cc_taste, pivot_gr_CC_taste_only, pivot_gr_CC_FamilyCode, pivot_gr_CC_SubFamliyCode], axis=1)

ppp.to_clipboard()

# Pivot concat 해보기

In [16]:
total_id_count = data.groupby('YYYYMM')['id'].count().reset_index(name='total_id_count')

gender_age_count = data.groupby(['YYYYMM', 'gender', 'age']).size().reset_index(name='count')

gender_pivot = gender_age_count.pivot_table(
    index ='YYYYMM',
    columns= ['gender', 'age'],
    values='count'
)

res = total_id_count.set_index('YYYYMM').join(gender_pivot)

res

MergeError: Not allowed to merge between different levels. (1 levels on the left, 2 on the right)

In [42]:
# -- Gr Region 집계 user_current_type_M1

gr_sql1= ''' 
with temp as (
select  
	t.YYYYMM, 
	gr_cd,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from  cu.agg_CU_TEREA_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM = t.YYYYMM
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    	
	t.YYYYMM, 
	gr_cd,
	t.id
)
select YYYYMM, gr_cd,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
     as Cigatype,
    count(*) purchaser_cnt
from temp
group by YYYYMM, gr_cd,
    'IQOS' +
    CASE WHEN CompHnB_Purchased = 1 THEN ' + Comp. HnB' ELSE '' END + 
    CASE WHEN CC_Purchased = 1 THEN ' + CC' ELSE '' END 
;
'''


gr_data = pd.read_sql_query(sql=gr_sql1, con=engine)

In [49]:
pivot_current_type = gr_data.pivot_table(index=['YYYYMM','gr_cd'],
                                       columns='Cigatype',
                                       values='purchaser_cnt',
                                       ).sort_values(['YYYYMM', 'gr_cd'], ascending=[True, False])

print(pivot_current_type)
pivot_current_type.to_clipboard()

Cigatype        IQOS  IQOS + CC  IQOS + Comp. HnB  IQOS + Comp. HnB + CC
YYYYMM gr_cd                                                            
202401 서울      885.0     1274.0             135.0                  190.0
       부산      337.0      430.0              44.0                   90.0
       대전      247.0      373.0              34.0                   68.0
       대구      168.0      283.0              28.0                   47.0
       광주      279.0      347.0              46.0                   74.0
       Other   254.0      385.0              50.0                   66.0
202402 서울      855.0     1280.0             142.0                  184.0
       부산      345.0      493.0              55.0                   68.0
       대전      257.0      405.0              47.0                   64.0
       대구      214.0      261.0              27.0                   45.0
       광주      246.0      406.0              46.0                   59.0
       Other   248.0      441.0              35.0  

In [43]:
gr_sql2 = ''' 
select  
	t.YYYYMM, 
	gr_cd,
	t.id,
	max(case when b.cigatype='HnB' and b.company = 'PMK' then 1 else 0 end) IQOS_Purchased,
	max(case when b.cigatype='CC' then 1 else 0 end) CC_Purchased,
	max(case when b.cigatype='HnB' and b.company != 'PMK' then 1 else 0 end) CompHnB_Purchased
from cu.agg_CU_TEREA_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and CIGADEVICE =  'CIGARETTES' AND  b.cigatype != 'CSV' 
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 	    
	t.YYYYMM, 
	gr_cd, 
	t.id
;
'''

gr_data2 = pd.read_sql_query(sql=gr_sql2, con=engine)

In [52]:
# Past Type 
def categorize(row):
    categories = []
    if row['IQOS_Purchased'] == 1:
        categories.append('PMK HnB')
    if row['CC_Purchased'] == 1:
        categories.append('CC')
    if row['CompHnB_Purchased'] == 1:
        categories.append('Comp HnB')
    return ','.join(categories)

gr_data2['retype'] = gr_data2.apply(categorize, axis=1)
result = gr_data2.groupby(['YYYYMM','gr_cd', 'retype']).size().reset_index(name='N')
total = result['N'].sum()

print(result)

     YYYYMM  gr_cd               retype    N
0    202401  Other                   CC  467
1    202401  Other          CC,Comp HnB  102
2    202401  Other             Comp HnB   28
3    202401  Other              PMK HnB   81
4    202401  Other           PMK HnB,CC   54
..      ...    ...                  ...  ...
247  202406     서울             Comp HnB  214
248  202406     서울              PMK HnB  330
249  202406     서울           PMK HnB,CC  222
250  202406     서울  PMK HnB,CC,Comp HnB   32
251  202406     서울     PMK HnB,Comp HnB   25

[252 rows x 4 columns]


In [56]:
pivot_past_type = result.pivot_table( 
                            index=['YYYYMM', 'gr_cd'],
                            columns='retype',
                            values='N'
                            ).sort_values(['YYYYMM', 'gr_cd'], ascending=[True, False])
print(pivot_past_type)
pivot_past_type.to_clipboard()

retype            CC  CC,Comp HnB  Comp HnB  PMK HnB  PMK HnB,CC  \
YYYYMM gr_cd                                                       
202401 서울     1535.0        267.0     141.0    296.0       202.0   
       부산      565.0        105.0      49.0    103.0        64.0   
       대전      484.0         66.0      43.0     75.0        48.0   
       대구      342.0         64.0      40.0     41.0        33.0   
       광주      468.0         84.0      40.0     86.0        58.0   
       Other   467.0        102.0      28.0     81.0        54.0   
202402 서울     1548.0        258.0     164.0    275.0       177.0   
       부산      605.0        102.0      59.0    102.0        74.0   
       대전      504.0        108.0      35.0     61.0        47.0   
       대구      346.0         62.0      38.0     59.0        36.0   
       광주      492.0         89.0      42.0     78.0        46.0   
       Other   518.0        110.0      39.0     74.0        54.0   
202403 서울     1717.0        283.0     172.0    2

In [58]:
# -- TEREA flavorXtar from 202211

gr_sql3 = ''' 
select  
	t.YYYYMM,
	gr_cd,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) flavorXtar,
	count(distinct case when b.cigatype ='CC' then t.id end) CC,
	count(distinct case when b.cigatype ='HnB' then t.id end) HnB
from  cu.agg_CU_TEREA_Sourcing t
	join cu.Fct_BGFR_PMI_Monthly a on t.id = a.id 
		and a.YYYYMM BETWEEN CONVERT(NVARCHAR(6), DATEADD(MONTH, -3, t.YYYYMM+'01'), 112)
				 	     AND CONVERT(NVARCHAR(6), DATEADD(MONTH, -1, t.YYYYMM+'01'), 112)	
	join cu.dim_product_master b on a.ITEM_CD = b.PROD_ID and b.CIGADEVICE =  'CIGARETTES' AND b.cigatype != 'CSV'  
	join cu.dim_Regional_area c on t.SIDO_nm = c.sido_nm
group BY 
	t.YYYYMM,
	gr_cd,
	concat(FLAVORSEG_type3,' X ', New_TARSEGMENTAT) 
;
'''

gr_data3 = pd.read_sql_query(sql=gr_sql3, con=engine)

In [60]:
pivot_flavor_tar = gr_data3.pivot_table(index=['YYYYMM', 'gr_cd'], 
                                     columns='flavorXtar',
                                     values=['HnB', 'CC' ]
                                    ).sort_values(['YYYYMM', 'gr_cd'], ascending=[True,True])
print(pivot_flavor_tar)
pivot_flavor_tar.to_clipboard()

                   CC                                                        \
flavorXtar   Fresh X  Fresh X 1MG Fresh X Below 1MG Fresh X LTS Fresh X ULT   
YYYYMM gr_cd                                                                  
202401 Other      0.0        35.0               NaN        41.0        15.0   
       광주         0.0        24.0               NaN        26.0         5.0   
       대구         0.0        16.0               NaN        17.0         6.0   
       대전         0.0        20.0               NaN        27.0         8.0   
       부산         0.0        28.0               NaN        24.0         2.0   
       서울         0.0       117.0               NaN       128.0        18.0   
202402 Other      0.0        36.0               NaN        44.0         8.0   
       광주         0.0        31.0               NaN        26.0         3.0   
       대구         0.0        19.0               NaN        12.0         4.0   
       대전         0.0        34.0               NaN 